In [93]:
class Node(object):
    def __init__(self, value):
        self.value = value
        self.edges = []
        self.visited = False
    
class Edge(object):
    def __init__(self, value, node_from, node_to):
        self.value = value
        self.node_from = node_from
        self.node_to = node_to
        
class Graph(object):
    def __init__(self, nodes = None, edges = None):
        self.nodes = nodes or []   # Là một list chứa các Nodes
        self.edges = edges or []   # Là một list chứa các Edges
        self.node_names = []       # Là một list chứa các names với index chính bằng giá trị của node đó
        self._node_map = {}        # Là một dict chứa các {node_value:node}
    
    def set_node_names(self, names):
        self.node_names = list(names)
        
    def insert_node(self, new_node_val):
        new_node = Node(new_node_val)
        self.nodes.append(new_node)
        self._node_map[new_node_val] = new_node
        return new_node
    
    def insert_edge(self, new_edge_val, node_from_val, node_to_val):
        "Insert a new edge, creating new nodes if necessary"
        nodes = {node_from_val: None, node_to_val: None}
        '''
        Chạy qua các node trong graph để kiếm node_from và node_to cho
        edge mới thêm vào nêu đã tìm được cả 2 thì break luôn.
        '''
        for node in self.nodes:
            if node.value in nodes:
                nodes[node.value] = node
                if all(nodes.values()):
                    break
        '''
        Xét 2 node trong nodes:
        Nếu chưa có node nào mang giá trị node_val thì sẽ self.insert 
        thêm 1 node với vào graph.
        '''
        for node_val in nodes:
            nodes[node_val] = nodes[node_val] or self.insert_node(node_val)
        
        '''
        Sau khi đã có được 2 node trong Graph thì chúng ta gắn cạnh mới
        này vào cho chúng cũng như cho Graph
        '''
        node_from = nodes[node_from_val]
        node_to = nodes[node_to_val]
        new_edge = Edge(new_edge_val, node_from, node_to)
        node_from.edges.append(new_edge)
        node_to.edges.append(new_edge)
        self.edges.append(new_edge)
        
    def get_edge_list(self):
        return [(e.value, e.node_from.value, e.node_to.value) for e in self.edges]
    
    def get_edge_list_names(self):
        return [(self.node_names[e.node_from.value],
                 self.node_names[e.node_to.value],
                 e.value)
                for e in self.edges]
    
    def find_max_index(self):
        '''
        Trả về số lượng node nhiều nhất tìm được,
        hoặc trả về chiều dài của list node names (tức là số tên node)
        có trong Graph nếu có set names cho các node.
        '''
        if len(self.node_names) > 0:
            return len(self.node_names)
        '''
        Nếu không set node names thì chạy hết các node trong Graph
        và nếu có node nào giá trị cao hơn max index thì gắn lại 
        max index.
        
        Tại sao lại phải làm như thế này cho mất công?
        Vì khi bạn tìm cái adjacency list hoặc matrix, nếu mà chỉ giựa
        vào số node trong Graph thì chắc chắn bạn sẽ phải mở rộng cái
        adjacency list hay matrix đấy khi insert thêm 1 node mới.
        
        (chẳng hạn lúc đầu có node 1, 5) -> insert thêm node 2, 3, 4
        
        Còn nếu ngay từ lúc đầu bạn dựa vào giá trị cao nhất của 
        các node là 5 thì ma trận adjacency của bạn sẽ không phải thay đổi.
        '''
        max_index = -1
        if len(self.nodes):
            for node in self.nodes:
                if node.value > max_index:
                    max_index = node.value
        return max_index
        
    def get_adjacency_list(self):
        '''
        Tìm giá trị cao nhất của các node sau đó tạo 1 cái list rỗng
        chiều dài bằng giá trị tìm được
        
        Chạy qua các edge trong Graph, lấy giá trị của node from và
        node to của edge.
        Gắn các giá trị vừa tìm được vào đúng index của list rỗng vừa tạo
        
        Thay các list trống [] bằng None
        '''
        max_index = self.find_max_index()
        adjacency_list = [[] for _ in range(max_index)]
        for edge in self.edges:
            from_value, to_value = edge.node_from.value, edge.node_to.value
            adjacency_list[from_value].append((to_value, edge.value))
        return [a or None for a in adjacency_list] # Thay các [] bằng None
    
    def get_adjacency_list_names(self):
        '''
        Get adjacency list
        
        Chạy qua các tuple trong adjacency list và map cái tuple này.
        Mapping:
            Nếu một tuple bằng None thì trả về None
            Nếu không thì trả về 1 object mới mà các giá trị trong tuple
            là 1 pair được đưa vào hàm convert trả về  giá trị của to_node
            cùng với độ lớn của edge sau đó sử dụng giá trị của to_node
            để  lấy name của to_node
        '''
        adjacency_list = self.get_adjacency_list()
        
        def convert_to_names(pair, graph = self):
            node_number, value = pair
            return (graph.node_names[node_number], value)
        def map_conversion(adjacency_list_for_node):
            if adjacency_list_for_node is None:
                return None
            '''
            Hàm map cho các iterable object vào 1 hàm nào đó và nhận
            giá trị trả về của hàm đó.
            '''
            return map(convert_to_names, adjacency_list_for_node)
        "Trả về một mảng với các phần tử được trả về sau khi qua hàm map"
        return [map_conversion(adjacency_list_for_node)()
                for adjacency_list_for_node in adjacency_list]
    
    def get_adjacency_matrix(self):
        max_index = self.find_max_index()
        adjacency_matrix = [[0]*(max_index) for _ in range(max_index)]
        for edge in self.edges:
            from_index, to_index = edge.node_from.value, edge.node_to.value
            adjacency_matrix[from_index][to_index] = edge.value
        return adjacency_matrix
    
    def find_node(self, node_value):
        "Trả về node có giá trị node_value hoặc None nếu node non exist"
        return self._node_map.get(node_value)
    
    def dfs(self, start_node_num):
        self._clear_visited() # Mark lại tất cả các node đều chưa visited
        start_node = self.find_node(start_node_num) # tìm node có giá trị start_node_num
        node_nums = self.dfs_helper(start_node) # trả về 1 list traversal chứa các node value
        return [self.node_names[num] for num in node_nums]
    
    def _clear_visited(self):
        for node in self.nodes:
            node.visited = False
    
    def dfs_helper(self, start_node):
        '''
        Phần tử đầu tiên của list traversal chính là giá trị node khởi đầu
        Gắn node khởi đầu là đã đi qua
        
        Xét các edge được gắn với node start đó, edge nào là edge có
        from_node là node start (hay to_node không phải start node) thì gắn
        edge đó vào list edges
        
        Duyệt qua các edge trong edges nếu qua edge có node nào chưa đc
        visit thì extend node đó là traversal list cũng như đệ quy với 
        start node là node vừa đến.
        
        Gọi là depth first thì luôn đệ quy đến node_to của 1 edge nào đó
        cho đến khi node_to không có edge đến 1 node khác thì quay trở lại node
        trước đó
        '''
        traversal_list = [start_node.value]
        start_node.visited = True
        edges = [e for e in start_node.edges 
                 if e.node_to.value != start_node.value]
        for edge in edges:
            if not edge.node_to.visited:
                traversal_list.extend(self.dfs_helper(edge.node_to))
        return traversal_list
    
    def bfs(self, start_node_num):
        
        node = self.find_node(start_node_num)
        self._clear_visited()
        traversal_list = []
        queue = [node]
        '''
        Hàm này sẽ append 1 node vao queue cũng như mark node đó là 
        đã visit.
        '''
        def enqueue(node, q = queue):
            node.visited = True
            q.append(node)
        
        '''
        Hàm này sẽ xét xem 1 node nào đó đã được vist chưa qua 1 edge
        cụ thể.
        '''
        def unvisited_outgoing_edge(node, edge):
            return ((edge.node_from.value ==  node.value) and 
                    (not edge.node_to.visited))
        '''
        Khi mà queue vẫn còn node:
            Lấy node đầu tiên ra append giá trị và traversal và duyệt:
                Qua hết các cạnh gắn với node, nếu có node nào chưa
                được đi đến từ node và edge đang xét thì append node
                node đó và queue.
        
        Được gọi là breadth first vì mỗi lần xét đến 1 node là lại đi qua
        hết các edge của node đó trước rồi mới đến node tiếp theo (vì sử dụng queue, pop)
        Khác với depth ở chỗ depth thì luôn đến node sâu nhất từ một node nhất định
        rồi mới trở lại node khi đã đến node sâu nhất.
        '''
        while queue:
            node = queue.pop(0)
            traversal_list.append(node.value)
            for edge in node.edges:
                if unvisited_outgoing_edge(node, edge):
                    enqueue(edge.node_to)
        return [self.node_names[num] for num in traversal_list[:-1]]

In [94]:
graph = Graph()
graph.set_node_names(('Mountain View',   # 0
                      'San Francisco',   # 1
                      'London',          # 2
                      'Shanghai',        # 3
                      'Berlin',          # 4
                      'Sao Paolo',       # 5
                      'Bangalore'))      # 6 

In [95]:
graph.insert_edge(51, 0, 1)     # MV <-> SF
graph.insert_edge(51, 1, 0)     # SF <-> MV
graph.insert_edge(9950, 0, 3)   # MV <-> Shanghai
graph.insert_edge(9950, 3, 0)   # Shanghai <-> MV
graph.insert_edge(10375, 0, 5)  # MV <-> Sao Paolo
graph.insert_edge(10375, 5, 0)  # Sao Paolo <-> MV
graph.insert_edge(9900, 1, 3)   # SF <-> Shanghai
graph.insert_edge(9900, 3, 1)   # Shanghai <-> SF
graph.insert_edge(9130, 1, 4)   # SF <-> Berlin
graph.insert_edge(9130, 4, 1)   # Berlin <-> SF
graph.insert_edge(9217, 2, 3)   # London <-> Shanghai
graph.insert_edge(9217, 3, 2)   # Shanghai <-> London
graph.insert_edge(932, 2, 4)    # London <-> Berlin
graph.insert_edge(932, 4, 2)    # Berlin <-> London
graph.insert_edge(9471, 2, 5)   # London <-> Sao Paolo
graph.insert_edge(9471, 5, 2)   # Sao Paolo <-> London

In [96]:
graph.get_edge_list_names()

[('Mountain View', 'San Francisco', 51),
 ('San Francisco', 'Mountain View', 51),
 ('Mountain View', 'Shanghai', 9950),
 ('Shanghai', 'Mountain View', 9950),
 ('Mountain View', 'Sao Paolo', 10375),
 ('Sao Paolo', 'Mountain View', 10375),
 ('San Francisco', 'Shanghai', 9900),
 ('Shanghai', 'San Francisco', 9900),
 ('San Francisco', 'Berlin', 9130),
 ('Berlin', 'San Francisco', 9130),
 ('London', 'Shanghai', 9217),
 ('Shanghai', 'London', 9217),
 ('London', 'Berlin', 932),
 ('Berlin', 'London', 932),
 ('London', 'Sao Paolo', 9471),
 ('Sao Paolo', 'London', 9471)]

In [97]:
graph.get_adjacency_list()

[[(1, 51), (3, 9950), (5, 10375)],
 [(0, 51), (3, 9900), (4, 9130)],
 [(3, 9217), (4, 932), (5, 9471)],
 [(0, 9950), (1, 9900), (2, 9217)],
 [(1, 9130), (2, 932)],
 [(0, 10375), (2, 9471)],
 None]

In [98]:
graph.dfs(2)

['London', 'Shanghai', 'Mountain View', 'San Francisco', 'Berlin', 'Sao Paolo']

In [99]:
graph.bfs(2)

['London', 'Shanghai', 'Berlin', 'Sao Paolo', 'Mountain View', 'San Francisco']